<a href="https://colab.research.google.com/github/fourfeatherz/DS2002F24/blob/main/Labs/Universities_and_Stock_API_Workshop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Stock Data Functions

In [77]:
import requests
import json
import pandas as pd
import matplotlib.pyplot as plt

stock=input("Please provide the ticker of the stock you want information on: ")

apikey="personal_key"

def get_stock_data(ticker):
    url = "https://yfapi.net/v6/finance/quote"

    querystring = {"symbols":ticker}

    headers = {
        'x-api-key': apikey
        }

    response = requests.request("GET", url, headers=headers, params=querystring)
    if response.status_code == 200:
        stock_json = response.json()
        outlist={"Ticker Name":[(stock_json['quoteResponse']['result'][0]["symbol"])], "Full Name":[(stock_json['quoteResponse']['result'][0]["longName"])], "Market Price":[(stock_json['quoteResponse']['result'][0]["regularMarketPrice"])], "52 Week High":[(stock_json['quoteResponse']['result'][0]["fiftyTwoWeekHigh"])], "52 Week Low":[(stock_json['quoteResponse']['result'][0]["fiftyTwoWeekLow"])]}
        return outlist
    else:
        return str(f"Failed to retrieve data: {response.status_code}")


def get_target_mean_price(ticker):
    url = f"https://yfapi.net/v11/finance/quoteSummary/{ticker}?lang=en&region=US&modules=financialData"

    querystring = {"symbols":ticker}

    headers = {
        'x-api-key': apikey
        }

    response = requests.request("GET", url, headers=headers, params=querystring)
    if response.status_code == 200:
        stock_json = response.json()
        temp = stock_json['quoteSummary']['result'][0]['financialData']['targetMeanPrice']
        if len(temp) != 0:
            return {"Target Mean Price":[temp['raw']]}
        else:
            return "There is no information on target mean price available."
    else:
        return str(f"Failed to retrieve data: {response.status_code}")
                   
def get_trending(ticker):
    url = f"https://yfapi.net/v1/finance/trending/US"

    querystring = {"symbols":ticker}

    headers = {
        'x-api-key': apikey
        }

    response = requests.request("GET", url, headers=headers, params=querystring)
    if response.status_code == 200:
        stock_json = response.json()
        if stock_json['finance']['result'][0]['count'] >= 5:
            outlist = stock_json['finance']['result'][0]['quotes'][:5]
            symbols = []
            for dict in outlist:
                symbols.append(dict['symbol'])
            outlist = [' '.join(symbols)] 
        elif stock_json['finance']['result'][0]['count'] > 0:
            num = stock_json['finance']['result'][0]['count']
            outlist = stock_json['finance']['result'][0]['quotes'][:num]
            symbols = []
            for dict in outlist:
                symbols.append(dict['symbol'])
            outlist = [' '.join(symbols)]
        else:
            return "There is no stocks currently trending."
        outlist = {"Trending Stocks":outlist}
        return outlist
    else:
        return str(f"Failed to retrieve data: {response.status_code}")

combined_data = get_stock_data(stock)
combined_data['Target Mean Price']=get_target_mean_price(stock)['Target Mean Price']
combined_data['Trending Stocks']=get_trending(stock)['Trending Stocks']
combined_data

Please provide the ticker of the stock you want information on: AAPL


{'Ticker Name': ['AAPL'],
 'Full Name': ['Apple Inc.'],
 'Market Price': [227.725],
 '52 Week High': [237.23],
 '52 Week Low': [164.08],
 'Target Mean Price': [240.58],
 'Trending Stocks': ['NQ=F ES=F YM=F AMSC GC=F']}

### Convert to DataFrame and CSV file

In [83]:
df = pd.DataFrame(combined_data)
df.to_csv(f"{stock}_stock_data.csv", index=False)
df

,Ticker Name,Full Name,Market Price,52 Week High,52 Week Low,Target Mean Price,Trending Stocks
0,AAPL,Apple Inc.,227.725,237.23,164.08,240.58,NQ=F ES=F YM=F AMSC GC=F
